In [1]:
case class LogRecord( host: String, timeStamp: String, url:String,httpCode:Int)

defined class LogRecord


In [2]:
val PATTERN = """^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)(.*)" (\d{3}) (\S+)""".r

PATTERN = ^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)(.*)" (\d{3}) (\S+)


^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)(.*)" (\d{3}) (\S+)

In [3]:
def parseLogLine(log: String):
	LogRecord = {
		val res = PATTERN.findFirstMatchIn(log) 
		if (res.isEmpty)
		{
			println("Rejected Log Line: " + log)
			LogRecord("Empty", "", "",  -1 )
		}
		else 
		{
			val m = res.get
			LogRecord(m.group(1), m.group(4),m.group(6), m.group(8).toInt)
		}
	}


parseLogLine: (log: String)LogRecord


In [4]:
val logFile = sc.textFile("/data/spark/project/NASA_access_log_Aug95.gz")
val accessLog = logFile.map(parseLogLine)
val accessDf = accessLog.toDF()

logFile = /data/spark/project/NASA_access_log_Aug95.gz MapPartitionsRDD[1] at textFile at <console>:33
accessLog = MapPartitionsRDD[2] at map at <console>:34
accessDf = [host: string, timeStamp: string ... 2 more fields]


[host: string, timeStamp: string ... 2 more fields]

Rejected Log Line: 203.16.174.5 - - [14/Aug/1995:04:12:17 -0400] "GET  HTTP/1.0" 302 -
Rejected Log Line: 128.159.144.83 - - [14/Aug/1995:15:51:37 -0400] "��6��T7�F��F" 400 -
Rejected Log Line: 128.159.144.83 - - [14/Aug/1995:15:51:58 -0400] "��6��T7�F��F" 400 -
Rejected Log Line: 128.159.144.83 - - [14/Aug/1995:15:52:23 -0400] "��6��T7�F��F" 400 -
Rejected Log Line: 128.159.144.83 - - [14/Aug/1995:15:59:35 -0400] "��6��T7�F��F" 400 -
Rejected Log Line: bloor.torfree.net - - [22/Aug/1995:14:55:16 -0400] "huttle/countdown/" 400 -
Rejected Log Line: mac391s.ksc.nasa.gov - - [23/Aug/1995:10:35:20 -0400] "GET  HTTP/1.0" 302 -
Rejected Log Line: mac391s.ksc.nasa.gov - - [23/Aug/1995:10:35:21 -0400] "GET  HTTP/1.0" 302 -
Rejected Log Line: mac391s.ksc.nasa.gov - - [23/Aug/1995:10:35:22 -0400] "GET  HTTP/1.0" 302 -
Rejected Log Line: mac391s.ksc.nasa.gov - - [23/Aug/1995:10:35:27 -0400] "GET  HTTP/1.0" 302 -
Rejected Log Line: mac391s.ksc.nasa.gov - - [23/Aug/1995:10:3

In [5]:
accessDf.printSchema

root
 |-- host: string (nullable = true)
 |-- timeStamp: string (nullable = true)
 |-- url: string (nullable = true)
 |-- httpCode: integer (nullable = true)



In [6]:
accessDf.createOrReplaceTempView("nasalog")

In [7]:
val output = spark.sql("select * from nasalog")

output = [host: string, timeStamp: string ... 2 more fields]


[host: string, timeStamp: string ... 2 more fields]

In [8]:
output.createOrReplaceTempView("nasa_log")

In [9]:
spark.sql("cache TABLE nasa_log")

[]

In [10]:
spark.sql("select url,count(*) as req_cnt from nasa_log where upper(url) like '%HTML%' group by url order by req_cnt desc LIMIT 10").show

[Stage 3:====================================================>  (192 + 8) / 200]+--------------------+-------+
|                 url|req_cnt|
+--------------------+-------+
|           /ksc.html|  43687|
|/shuttle/missions...|  24606|
|/shuttle/missions...|  22453|
|/software/winvn/w...|  10345|
|/history/history....|  10134|
|/history/apollo/a...|   8985|
|/shuttle/countdow...|   7865|
|/history/apollo/a...|   7177|
|/shuttle/technolo...|   6517|
|/shuttle/missions...|   5264|
+--------------------+-------+



In [11]:
spark.sql("select host,count(*) as req_cnt from nasa_log group by host order by req_cnt desc LIMIT 5").show

+--------------------+-------+
|                host|req_cnt|
+--------------------+-------+
|  edams.ksc.nasa.gov|   6530|
|piweba4y.prodigy.com|   4846|
|        163.206.89.4|   4791|
|piweba5y.prodigy.com|   4607|
|piweba3y.prodigy.com|   4416|
+--------------------+-------+



In [12]:
spark.sql("select substr(timeStamp,1,14) as timeFrame,count(*) as req_cnt from nasa_log group by substr(timeStamp,1,14) order by req_cnt desc LIMIT 5").show

+--------------+-------+
|     timeFrame|req_cnt|
+--------------+-------+
|31/Aug/1995:11|   6319|
|31/Aug/1995:10|   6283|
|31/Aug/1995:13|   5948|
|30/Aug/1995:15|   5919|
|31/Aug/1995:09|   5627|
+--------------+-------+



In [13]:
spark.sql("select substr(timeStamp,1,14) as timeFrame,count(*) as req_cnt from nasa_log group by substr(timeStamp,1,14) order by req_cnt  LIMIT 5").show

+--------------+-------+
|     timeFrame|req_cnt|
+--------------+-------+
|03/Aug/1995:04|     16|
|03/Aug/1995:09|     22|
|03/Aug/1995:05|     43|
|03/Aug/1995:10|     57|
|03/Aug/1995:07|     58|
+--------------+-------+



In [14]:
spark.sql("select httpCode,count(*) as req_cnt from nasa_log group by httpCode ").show

+--------+-------+
|httpCode|req_cnt|
+--------+-------+
|      -1|     63|
|     501|     27|
|     500|      3|
|     403|    171|
|     404|  10056|
|     200|1398988|
|     304| 134146|
|     302|  26444|
+--------+-------+

